In [378]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from tqdm.notebook import tqdm
import time
import sys

In [ ]:
driver = webdriver.Chrome(executable_path = r'C:\Users\Administrator\Downloads\chromedriver.exe')

In [ ]:
df1 = pd.read_excel(r'ASINS_Flipkart.xlsx',header = None,engine = 'openpyxl')

In [ ]:
# Need to get catalog images
# driver.get('https://www.flipkart.com/product/p/itme?pid=MOBGYFAXFPHUPEE5')
def getCatalogContent(driver):
    
    imgs = driver.find_elements_by_css_selector('div>ul>li>div>div[style*=background-image]')
    numVids = len(driver.find_elements_by_css_selector('div>ul>li>div>div:nth-child(3)'))
    imgList = []
    for i in imgs:
        url_ = i.value_of_css_property('background-image').strip('url("")').replace('128/','416/',2)
        imgList.append(url_)
    return imgList,numVids


In [294]:
# Need to get all the remaining info on the webPage including features, description and Product title



In [295]:
# product_title = driver.find_element_by_css_selector('div>div>div>h1').text
def getTechnicalDetails(driver):
    details_fs = driver.find_elements_by_css_selector('table')
    df_prodTabularDetails = pd.DataFrame()
    productDetails = {}
    for tl in details_fs:
        table_content = tl.get_attribute('innerHTML')
        # pd.read_html(table_content)
        df = pd.read_html('<table>' + table_content + '</table>')[0]
        df_prodTabularDetails = df_prodTabularDetails.append(df,ignore_index=True)
    productDetails = df_prodTabularDetails.set_index(0)[1].to_dict()
    return productDetails



In [374]:
def getFeatureDetails(driver):
    feature_button = driver.find_elements_by_xpath("//button[text()='View all features']")
    if(len(feature_button)!=0):
        feature_button[0].click()
    details_elements = driver.find_elements_by_css_selector('div.col-12-12 div:nth-child(2)>p')

    featureDetails = {}
    for ftrIdx in range(len(details_elements)):
        featureDetails['feature_' + str(ftrIdx)] = details_elements[ftrIdx].text
    return featureDetails


In [370]:
feature_button = driver.find_elements_by_xpath("//button[text()='View all features']")

In [297]:
def getHlEmiDetails(driver):
    OtherDetails = driver.find_elements_by_css_selector('div.col-6-12')
    hl_emi_details = {}
    for hl_emi_idx in range(len(OtherDetails)):
        _detail = OtherDetails[hl_emi_idx].text
        detail_pair = _detail.split('\n',maxsplit = 1)
        if(sample!='' and len(detail_pair)>1):
            hl_emi_details[detail_pair[0]] = detail_pair[1]
    return hl_emi_details

In [303]:
def getASINData(asin,timeout,locator,driver):
    link = 'https://www.flipkart.com/product/p/itme?pid=' + str(asin)

    driver.get(link)
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.CSS_SELECTOR, locator)))
    product_title = driver.find_element_by_css_selector('div>div>div>h1').text
    img_list,num_vids = getCatalogContent(driver)
    productDetails = getTechnicalDetails(driver)
    featureDetails = getFeatureDetails(driver)
    hl_emi_details =  getHlEmiDetails(driver)
    
    final_res = {}
    final_res['product title'] = product_title
    final_res['product details'] = productDetails
    final_res['feature details'] = featureDetails
    final_res['hl_emi details'] = hl_emi_details
    final_res['images'] = img_list
    final_res['num vids'] = num_vids
    return final_res

In [381]:
def getData(asin_list,asin_start_index,timeout,locator,driver):
    asin_page = {}
    current_asin_index = 0
    for asin_idx in tqdm(range(asin_start_index,len(asin_list))):
        try:
            asin = asin_list[asin_idx]
            asin_page[asin] = getASINData(asin,timeout,locator,driver)
            current_asin_index = asin_idx
        except KeyboardInterrupt:
            print('user paused loop')
            break
        except:
            print(sys.exc_info())
            pass
    # print(asin_page)
    return asin_page,current_asin_index


In [ ]:
dfASINList = pd.read_csv(r'C:\Users\Administrator\Downloads\query_result_2021-05-25T13_50_17.118447Z.csv')
asin_list = list(set(dfASINList['channel_sku_id']))

In [382]:
locator = 'div>div>div>h1'
timeout = 10
asin_start_index = 0 #change to asin_current_index to resume loop from the point where it was paused
# asin_start_index = asin_current_index #
asin_details = {} ## comment this line and update asin_details with asin_page before restarting loop
asin_page,asin_current_index = getData(asin_list,asin_start_index,timeout,locator,driver)
asin_details.update(asin_page)
print("asin details", asin_page)

  0%|          | 0/1367 [00:00<?, ?it/s]

es': 'Li-ion Battery', 'Warranty Summary': '1 Year Limited Hardware Warranty, In Home Service After Remote Diagnosis - Retail', 'Warranty Service Type': 'Onsite', 'Covered in Warranty': 'Manufacturing Defects', 'Not Covered in Warranty': 'Physical Damage', 'Domestic Warranty': '1 Year'}, 'feature details': {}, 'hl_emi details': {'Highlights': 'Pre-installed Genuine Windows 10 OS\nPreloaded with MS Office\nLight Laptop without Optical Disk Drive\n15.6 inch Full HD LED Backlit Truelife Display', 'Easy Payment Options': 'No cost EMI starting from ₹7,958/month\nNet banking & Credit/ Debit/ ATM card\nView Details'}, 'images': ['https://rukminim1.flixcart.com/image/416/416/k5r2mq80/computer/d/h/n/dell-na-thin-and-light-laptop-original-imafzb2hjz6x7zb7.jpeg?q=70', 'https://rukminim1.flixcart.com/image/416/416/k5r2mq80/computer/d/h/n/dell-na-thin-and-light-laptop-original-imafzd3jke4ucmbk.jpeg?q=70', 'https://rukminim1.flixcart.com/image/416/416/k5r2mq80/computer/d/h/n/dell-na-thin-and-light-l

In [383]:
df_FlipkartData = pd.DataFrame.from_dict(asin_page,orient = 'index')

In [386]:
df_FlipkartData.to_csv('flipkart_data.csv')

In [387]:
df_FlipkartData

,product title,product details,feature details,hl_emi details,images,num vids
PRNEZ6UHEJVC5DYZ,HP LaserJet Pro M403n Single Function Monochro...,"{'Printing Method': 'Laser', 'Type': 'Single F...",{},{'Highlights': 'Laser Output: Monochrome USB'},[https://rukminim1.flixcart.com/image/416/416/...,0
MIXEMXG9TUWDRZA8,Sujata POWERMATIC PLUS 900 Juicer Mixer Grinde...,"{'Sales Package': '1 JUICER, 1 BLENDER, 1 GRIN...",{},"{'Total Jars': '2', 'Highlights': 'Centrifugal...",[],0
PSLFWWXT8YBCHZEA,BCS Mass Mass Trix 11 lbs Weight Gainers/Mass ...,"{'Brand': 'BCS', 'Model Number': 'MHTRX2003TRI...",{},"{'Quantity': '5 kg', 'Flavor': 'Chocolate', 'H...",[https://rukminim1.flixcart.com/image/416/416/...,0
PSLFNHY92EP4MP5S,Osoaa The Real Unflavoured Whey Protein Concen...,"{'Brand': 'Osoaa', 'Model Number': 'unflv-why-...",{},"{'Quantity': '1 kg 2 kg', 'Flavor': 'Unflavore...",[https://rukminim1.flixcart.com/image/416/416/...,0
COMFWHGBDHP4FJMR,Lenovo Core i3 10th Gen - (4 GB/256 GB SSD/DOS...,"{'Sales Package': 'Laptop, Power Adaptor, User...",{},{'Highlights': 'Light Laptop without Optical D...,[https://rukminim1.flixcart.com/image/416/416/...,0
...,...,...,...,...,...,...
PSLET4NDF84FHZGV,"MUSCLEBLAZE Raw Whey Protein (1 kg, Unflavored)","{'Sales Package': '1 jar of Raw Whey Protein, ...",{},"{'Quantity': '500 g 1 kg 2 kg 4 kg', 'Flavor':...",[https://rukminim1.flixcart.com/image/416/416/...,1
PSLFAVME2FXF7AFM,"Labrada Whey Protein (500 g, Vanilla)","{'Brand': 'Labrada', 'Model Number': '71077911...",{},"{'Quantity': '500 g 1 kg 1875 g 2 kg', 'Flavor...",[https://rukminim1.flixcart.com/image/416/416/...,0
PRNF6M6F8V9BPNQ5,HP ink tank wireless 415 All in one Multi-func...,"{'Printing Method': 'Inkjet', 'Type': 'Multi-f...",{'feature_0': 'This HP printer comes with a la...,"{'Highlights': 'Ink Tank Output: Color WiFi, W...",[https://rukminim1.flixcart.com/image/416/416/...,0
EKTEJFFU4MBQDHAE,"Pigeon Kessel Multi Electric Kettle (1.2 L, B...","{'Model Name': '12173', 'Operating Mode': 'Cor...",{'feature_0': 'The wide mouth of this kettle m...,"{'Capacity': '1.2 L', 'Highlights': 'Suitable ...",[https://rukminim1.flixcart.com/image/416/416/...,0


In [324]:
asin_details.update(asin_page)

In [325]:
asin_details

 'Model Name': 'Whey Protein with 24 g protein, 5.4 g BCAA - 2 lbs / 907 g',
   'Product information provided by the seller on the Website is not exhaustive, please read the label on the physical product carefully for complete information provided by the manufacturer. For additional information, please contact the manufacturer.': nan},
  'feature details': {},
  'hl_emi details': {'Quantity': '1 pounds\n2 pounds',
   'Flavor': 'Chocolate\nCookies & Cream\nFigs & Honey\nMango\nPina Colada\nUnflavored',
   'Highlights': 'Form: Powder\nVegetarian\nUsage Post-workout\nProtein Type: Whey Protein\nDietary Preference: Sugar Free',
   'Services': 'No Returns Applicable?\nCash on Delivery available?'},
  'images': ['https://rukminim1.flixcart.com/image/416/416/knxiavk0/protein-supplement/f/g/x/whey-protein-with-24-g-protein-5-4-g-bcaa-2-lbs-907-g-uth-bev-original-imag2hyjgqcbhfvv.jpeg?q=70',
   'https://rukminim1.flixcart.com/image/416/416/kokdci80/protein-supplement/h/5/t/whey-protein-with-24-

{'feature_0': 'If you are looking for a speedy and pocket-friendly personal laser printer, then this Brother laser printer is ideal for you. It boasts a speed of 30 ppm along with a printing resolution of 2400 x 600 dpi so that your printing experience is optimal and quick. Moreover, it comes equipped with the automatic duplex feature, which allows you to easily print on both sides and save paper while looking professional. Furthermore, it comes with an adjustable 250-sheet capacity paper tray for normal-sized papers and a manual feed for envelopes, letterheads and so on.',
 'feature_1': 'This Brother printer boasts a print resolution of up to 2400 x 600 dpi and a printing speed of up to 30 ppm, which save your time by delivering optimally printed copies in a jiffy.',
 'feature_2': 'Look professional and save nature by using the automatic duplex printing feature, which allows you to print two-sided documents.',
 'feature_3': '',
 'feature_4': '',
 'feature_5': 'Wonderful',
 'feature_6'

In [ ]:
img_list,num_vids = getCatalogContent()
print("Images",img_list,"Number of videos",num_vids)